In [ ]:

import os
import requests
import openpyxl
import matplotlib.pyplot as plt
import win32com.client as win32

# Step 1: Make API call and retrieve JSON data
url = "https://eodhd.com/api/fundamentals/AAPL.US?api_token=demo&fmt=json"
response = requests.get(url)
data = response.json()

"""
# Save the JSON data to a file

import json

with open("AAPL_fundamentals.json", "w") as json_file:
    json.dump(data, json_file, indent=4)
"""

# Step 2: Extract relevant data for the last 10 years
years = [str(year) + '-09-30' for year in range(2013, 2024)]

net_profit_margin_data = []
current_ratio_data = []
roe_data = []

# Extract net profit margin data
yearly_income_statement = data['Financials']['Income_Statement']['yearly']
for year in years:
    net_income = float(yearly_income_statement.get(year, {}).get('netIncome', '0'))
    total_revenue = float(yearly_income_statement.get(year, {}).get('totalRevenue', '0'))
    net_profit_margin = (net_income / total_revenue) * 100 if total_revenue else 0
    net_profit_margin_data.append(net_profit_margin)

# Extract current ratio and ROE data
yearly_balance_sheet = data['Financials']['Balance_Sheet']['yearly']
for year in years:
    current_assets = float(yearly_balance_sheet.get(year, {}).get('totalCurrentAssets', '0'))
    current_liabilities = float(yearly_balance_sheet.get(year, {}).get('totalCurrentLiabilities', '0'))
    net_income = float(yearly_income_statement.get(year, {}).get('netIncome', '0'))
    shareholders_equity = float(yearly_balance_sheet.get(year, {}).get('totalStockholderEquity', '0'))
    
    current_ratio = current_assets / current_liabilities if current_liabilities else 0
    roe = (net_income / shareholders_equity) * 100 if shareholders_equity else 0
    
    current_ratio_data.append(current_ratio)
    roe_data.append(roe)

# Step 3: Create Excel workbook and add data
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Financial Data"

ws.append(["Year", "Net Profit Margin (%)", "Current Ratio", "ROE"])
for i, year in enumerate(years):
    ws.append([year, net_profit_margin_data[i], current_ratio_data[i], roe_data[i]])

# Step 4: Create charts
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Function to add data labels
def add_data_labels(ax, x_data, y_data):
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        ax.annotate(f'{y:.1f}', (x, y), textcoords="offset points", xytext=(0,5), ha='center')

# Net Profit Margin Chart
axs[0].plot(years, net_profit_margin_data, marker='o')
axs[0].set_title('Net Profit Margin (2013-2023)')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Net Profit Margin (%)')
axs[0].tick_params(axis='x', rotation=45)
add_data_labels(axs[0], years, net_profit_margin_data)

# Current Ratio Chart
axs[1].plot(years, current_ratio_data, marker='o')
axs[1].set_title('Current Ratio (2013-2023)')
axs[1].set_xlabel('Year')
axs[1].set_ylabel('Current Ratio')
axs[1].tick_params(axis='x', rotation=45)
add_data_labels(axs[1], years, current_ratio_data)

# ROE Chart
axs[2].plot(years, roe_data, marker='o')
axs[2].set_title('Return on Equity (ROE) (2013-2023)')
axs[2].set_xlabel('Year')
axs[2].set_ylabel('ROE (%)')
axs[2].tick_params(axis='x', rotation=45)
add_data_labels(axs[2], years, roe_data)

 # Save chart as PDF
plt.tight_layout()
pdf_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'financial_metrics.pdf')
plt.savefig(pdf_path) 

# Step 5: Save Excel file
excel_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'stockinvest.xlsx')
wb.save(excel_path)

# Step 6: Send email with PDF attachment using Microsoft Outlook
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.Subject = 'Financial Metrics Report'
mail.Body = 'Please find attached the financial metrics report for AAPL.'
mail.To = 'your email address'

attachment = pdf_path
mail.Attachments.Add(attachment)

mail.Send()

print(f'Email sent with attachment: {attachment}')
